## **Case Studies in Data Analytics Assignment 2**

Group Number: 1

Group Member Names:

1.   Shimita Rudra (22221035)
2.   Ayusha Kakkad (22210321)
3.   Sanju Kumari(22221634)








### Mount Drive

In [ ]:
# Mounting the dataset from google drive in colab.
from google.colab import drive
drive.mount('/content/drive')


### Import Packages and Libraries

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
%reload_ext tensorboard
import datetime


In [ ]:
# read the image folder path from goole drive

image_folder = '/content/drive/MyDrive/dataset/'

# set lables for prediction
lables = ["not alpaca", "alpaca"]

### Proposed Layer Model Implementation

#### Read Images and Convert to GreyScale

In [ ]:
# Read all the images from folder

image_dataset = []
X = []
y = []

for idx, i in enumerate(lables):
  path = os.path.join(image_folder,i)
  label_num = idx
  for image in os.listdir(path):
    #Convert to greyscale
    image_array = cv2.imread(os.path.join(path, image), cv2.IMREAD_GRAYSCALE)
    try:
      image_array = cv2.resize(image_array,(32,32),interpolation=cv2.INTER_AREA)
    except:
      break
    image_dataset.append([image_array,label_num])

for i, j in image_dataset:
  X.append(i)
  y.append(j)
y = np.array(y)
X=np.array(X).reshape(-1,32,32,1)

#### Train Test Split

In [ ]:
X_Train, X_Test, y_train, y_test= train_test_split(X, y, test_size=0.3, random_state=12) # 70% data for training
X_Val, X_Test, y_val, y_test= train_test_split(X_Test, y_test, test_size=0.5, random_state=12) # 15% data for test and validation
X_Train = tf.cast(X_Train, tf.float32)
X_Val = tf.cast(X_Val, tf.float32)
X_Test = tf.cast(X_Test, tf.float32)
X_Train.shape


#### Proposed Layer Method

In [ ]:
class ProposedLayer(tf.keras.layers.Layer):
  def __init__(self, depth_weight_matrics, kernel_size, **kwargs):
    self.depth_weight_matrics = depth_weight_matrics
    self.kernel_size = kernel_size
    super(ProposedLayer, self).__init__(**kwargs)

#Build Function to initialize weight matrix
  def build(self, input_shape):
    self.weight_matrix = self.add_weight(name='kernel',shape=(input_shape[1], input_shape[2], input_shape[-1], self.depth_weight_matrics),
                                  initializer='glorot_uniform',trainable=True)
    super(ProposedLayer, self).build(input_shape)

#Call Function
  def call(self, input):
    outputs = []
    #print("Input matrix", input[:,:,:])
    for k in range(0,self.depth_weight_matrics):
      #print(self.weight_matrix[:,:,0,k])
      #print("input :", input[:,:,:,0])
#Matrix Multiplication of Input and Weight Matrix
      x = tf.math.multiply(input[:,:,:,0], self.weight_matrix[:,:,0,k])
      x = tf.expand_dims(x,axis=-1)
      #print("x:",x.shape)
      #print(input,"\n", type(x),"\n", type(input))
      receptive_field = tf.cast(tf.constant(np.ones((3,3,1,1))), tf.float32)
      #print(receptive_field)
      #print("WEight mat :",type(self.weight_matrix),self.weight_matrix, "\n" ,self.weight_matrix[:,:,0,k])
      #print("X:", type(x),x.shape)

#tf.nn.conv2D function
      output = tf.nn.conv2d(x, receptive_field, strides = 1, padding="SAME")
      outputs.append(output)
      #print("output :", output.shape)
    outputs = tf.concat(outputs, axis=-1)
    #print("output shape",output.shape)
    return outputs

#Compute the gradients of the layer's
  def gradient(self, x, gradients):
    with tf.GradientTape() as tape:
      tape.watch()
      outputs = self.call(x)
    output_gradients = tape.gradient(outputs, x, gradients)
    weight_gradients = tape.gradient(outputs, self.weight_matrix, gradients)
    print(weight_gradients)
    return output_gradients, weight_gradients

#### Preprocessing

In [ ]:
#Augment data with RandomFlip,RandomRotation and RandomZoom of Images
custom_ccn_model = models.Sequential( [
    layers.RandomFlip("horizontal",
                      input_shape=(32,
                                  32,
                                  1)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ])

#### Create Sequential Model

In [ ]:
#Preprocessed Images put in Input Layer
custom_ccn_model.add(layers.InputLayer())

#Kernal Number = 16,Weighted Matrix Size = (3,3)
custom_ccn_model.add(ProposedLayer(depth_weight_matrics=16,kernel_size=3))

#Apply Relu Activation Function
custom_ccn_model.add(layers.Activation('relu'))

#Max Pooling Kernel Size = (2,2)
custom_ccn_model.add(layers.MaxPooling2D(pool_size=(2,2)))

#Kernal Number = 12,Weighted Matrix Size = (3,3)
custom_ccn_model.add(ProposedLayer(depth_weight_matrics=12,kernel_size=3))

#Apply Relu Activation Function
custom_ccn_model.add(layers.Activation('relu'))

#Max Pooling Kernel Size = (2,2)
custom_ccn_model.add(layers.MaxPooling2D(pool_size=(2,2)))

#Kernal Number = 16,Weighted Matrix Size = (3,3)
custom_ccn_model.add(ProposedLayer(depth_weight_matrics=8,kernel_size=3))

#Apply Relu Activation Function
custom_ccn_model.add(layers.Activation('relu'))

#Convert to 1 Dimensional
custom_ccn_model.add(layers.Flatten())

#Performing Linear Operation on input
custom_ccn_model.add(layers.Dense(256))

#Apply Relu Activation Function
custom_ccn_model.add(layers.Activation('relu'))

#Applying softmax activation to output result
custom_ccn_model.add(layers.Dense(1, activation = 'softmax'))
custom_ccn_model(X_Train)

#Print Model Summary
custom_ccn_model.summary()

#### Proposed Layer Model Compilation and Training

In [ ]:
#Stochastic Gradient Optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

#Compile Model Trained
custom_ccn_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['mse', 'accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

#Tensorboard callbacks
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#60 epochs
epo = 60

#Train the model
hist=custom_ccn_model.fit(X_Train, y_train,
                    epochs=epo, validation_data=(X_Val, y_val),batch_size = 32,callbacks=[tensorboard_callback1])

#### Proposed Layer Model Evaluation and Prediction

In [ ]:
#Evaluation on binary_crossentropy loss with mse,accuracy,precision,recall metrics
test_loss, test_mse, test_acc,  test_precision, test_recall = custom_ccn_model.evaluate(X_Test, y_test)

#Image Prediction
y_pred_new = custom_ccn_model.predict(X_Test)
y_pred_new = np.around(y_pred_new)


#### Training and Validation Loss Plot

In [ ]:
#X axis = Epochs
#Y axis = Loss value
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']

loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs_range = range(epo)
"""
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
"""

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss with learning rate 0.01,Epochs size 60 and batch size 32')
plt.show()


#### Tensorboard Implementation

In [ ]:
  !tensorboard dev upload --logdir ./logs \
  --name "Proposed Layer Model Implementation with SGD Optimizer" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot

### Proposed Layer Model Implementation with Different Hyperparameters

In [ ]:
#Learning rates - 0.01,0.001,0.05
#epochs(epo) - 200,400,50
#batch sizes - 8,16,32
#optimizer - adam

from matplotlib.ticker import NullFormatter
for i in (0.01,0.001,0.05):
  batch_sizes = ''
  epo = ''
  if i == 0.01:
    batch_sizes=8
    epo=200
    if i==0.001:
      batch_sizes=16
      epo=400
  else:
    batch_sizes=32
    epo=50

  optimizer = tf.keras.optimizers.Adam(learning_rate=i)
  custom_ccn_model.compile(optimizer=optimizer, loss='binary_crossentropy',
                           metrics=['mse', 'accuracy', tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback1 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

  hist=custom_ccn_model.fit(X_Train, y_train,
                    epochs=epo, validation_data=(X_Val, y_val), batch_size = batch_sizes,callbacks=[tensorboard_callback1])
  test_loss, test_mse, test_acc,  test_precision, test_recall = custom_ccn_model.evaluate(X_Test, y_test)

  y_pred_new = custom_ccn_model.predict(X_Test)
  y_pred_new = np.around(y_pred_new)

  acc = hist.history['accuracy']
  val_acc = hist.history['val_accuracy']

  loss = hist.history['loss']
  val_loss = hist.history['val_loss']

  epochs_range = range(epo)
  plt.subplot(1, 2, 2)
  plt.plot(epochs_range, loss, label='Training Loss')
  plt.plot(epochs_range, val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title("Training and Validation Loss with learning rate {} ,batch size {} and epoch size {}".format(i,batch_sizes,epo))
  #plt.title('Training and Validation Loss with learning rate :',i, )
  plt.show()
  !tensorboard dev upload --logdir ./logs \
  --name "Proposed Layer Model Implementation with Different Hyperparameters with Optimizer Adam" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot



### Traditional Convolution Neural Network without Data Augmentation

#### Import Packages and Libraries

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, Input, Dropout, RandomFlip, RandomRotation,Rescaling
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import datetime

#### Train Test Split

In [ ]:
Batch_size = 32
Image_height=32
Image_width=32

train_dataset = image_dataset_from_directory(image_folder,
                                             shuffle=True,
                                             batch_size=Batch_size,
                                             color_mode='grayscale',
                                             image_size=(Image_height,Image_width),
                                             validation_split=0.3,
                                             subset='training',
                                             seed=30)
val_dataset = image_dataset_from_directory(image_folder,
                                             shuffle=True,
                                             batch_size=Batch_size,
                                             color_mode='grayscale',
                                             image_size=(Image_height,Image_width),
                                             validation_split=0.3,
                                             subset='validation',
                                             seed=30)

class_names = train_dataset.class_names

num_classes = len(class_names)

for image_batch, labels_batch in train_dataset:
  print(image_batch.shape)
  print(labels_batch.shape)
  break


#### Create Sequential Model

In [ ]:
from keras.metrics.confusion_metrics import Precision

#Autotune to efficiently load data
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(100).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

ccn_model_1 = Sequential([
  #rescale input
  layers.Rescaling(1./255, input_shape=(32, 32, 1)),

  #Apply Relu Activation Function
  layers.Conv2D(16, 3, padding='same', activation='relu'),

  #Max Pooling Kernel Size = (2,2)
  layers.MaxPooling2D(),

  #Apply Relu Activation Function
  layers.Conv2D(12, 3, padding='same', activation='relu'),

  #Max Pooling Kernel Size = (2,2)
  layers.MaxPooling2D(),

  #Apply Relu Activation Function
  layers.Conv2D(8, 3, padding='same', activation='relu'),

  #Max Pooling Kernel Size = (2,2)
  layers.MaxPooling2D(),

  #Convert to 1 Dimensional
  layers.Flatten(),

  layers.Dense(256, activation='relu'),

  #Applying softmax activation to output result
  layers.Dense(num_classes, activation='softmax')
])


#### Traditional CNN Model Compilation and Training

In [ ]:
#Compile model
ccn_model_1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

#create model summary
ccn_model_1.summary()

#Tensorboard Callbacks
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback2 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

epochs=10

#Train model
ccn_model_1_hist = ccn_model_1.fit(
  train_dataset,
  validation_data=val_dataset,
  epochs=epochs,
  callbacks=[tensorboard_callback2]
)

#### Traditional CNN Model Prediction

In [ ]:
y_pred_new = ccn_model_1.predict(X_Test)
y_pred_new = np.around(y_pred_new)

#### Tensorboard Implementation

In [ ]:
#%tensorboard --logdir logs/fit

!tensorboard dev upload --logdir ./logs \
  --name "Traditional_CNN_without_DataAugmentation" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot

#### Training,Validation Accuracy and Loss Plot

In [ ]:
#X axis = Epochs
#Y axis = Loss, Accuracy value

acc = ccn_model_1_hist.history['accuracy']
val_acc = ccn_model_1_hist.history['val_accuracy']

loss = ccn_model_1_hist.history['loss']
val_loss = ccn_model_1_hist.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


#### Data Augmentation

In [ ]:
#Augment data with RandomFlip,RandomRotation and RandomZoom of Images
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(32,
                                  32,
                                  1)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

### Traditional CNN Model with Data Augmentation

In [ ]:

ccn_model_2 = Sequential([
  data_augmentation,
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(12, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(8, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

#### Model Compilation and Training

In [ ]:
ccn_model_2.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

ccn_model_2.summary()
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback3 = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

epochs = 10
ccn_model_2_hist = ccn_model_2.fit(
  train_dataset,
  validation_data=val_dataset,
  epochs=epochs,
  callbacks=[tensorboard_callback3]
)

####Tensorboard Implementation

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "Traditional_CNN_with_DataAugmentation" \
  --description "Training results from https://colab.sandbox.google.com/github/tensorflow/tensorboard/blob/master/docs/tbdev_getting_started.ipynb" \
  --one_shot

#### Training,Validation Accuracy and Loss Plot

In [ ]:
acc2 = ccn_model_2_hist.history['accuracy']
val_acc2 = ccn_model_2_hist.history['val_accuracy']

loss2 = ccn_model_2_hist.history['loss']
val_loss2 = ccn_model_2_hist.history['val_loss']

epochs_range = range(epochs)

In [ ]:
#X axis = Epochs
#Y axis = Loss,Accuracy value

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc2, label='Training Accuracy')
plt.plot(epochs_range, val_acc2, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss2, label='Training Loss')
plt.plot(epochs_range, val_loss2, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

### Proposed Layer Model for 1-Dimensional Data

In [ ]:
class ProposedLayer_1D(tf.keras.layers.Layer):
  def __init__(self, depth_weight_matrics, kernel_size, **kwargs):
    self.depth_weight_matrics = depth_weight_matrics
    self.kernel_size = kernel_size
    super(ProposedLayer_1D, self).__init__(**kwargs)

  def build(self, input_shape):
    #adjusted kernal size
    self.weight_matrix = self.add_weight(name='kernel', shape=(input_shape[-1], self.depth_weight_matrics),
                                  initializer='glorot_uniform', trainable=True)
    super(ProposedLayer_1D, self).build(input_shape)

  def call(self, input):
    outputs = []
    for k in range(0, self.depth_weight_matrics):
      x = tf.math.multiply(input[:, :, 0], self.weight_matrix[:, k])
      #convert to NLC format
      x = tf.expand_dims(x, axis=-1)
      receptive_field = tf.cast(tf.constant(np.ones((1, 1, 1, 1))), tf.float32)
      output = tf.nn.conv2d(x, receptive_field, strides=1, padding="SAME")
      outputs.append(output)
    outputs = tf.concat(outputs, axis=3)
    return outputs

### Proposed Layer Model for 3-Dimensional Data

In [ ]:
class ProposedLayer_3D(tf.keras.layers.Layer):
  #Adjusted kernel size
  def __init__(self, depth_weight_matrices, kernel_size=(3,3,3), **kwargs):
    self.depth_weight_matrices = depth_weight_matrices
    super(ProposedLayer_3D, self).__init__(**kwargs)

  def build(self, input_shape):
    self.weight_matrix = self.add_weight(name='kernel', shape=(self.kernel_size[0], self.kernel_size[1], self.kernel_size[2], input_shape[-1], self.depth_weight_matrices),
                                  initializer='glorot_uniform', trainable=True)
    super(ProposedLayer_3D, self).build(input_shape)

  def call(self, input):
    outputs = []
    for k in range(self.depth_weight_matrices):
      x = tf.math.multiply(input[:,:,:,0,:], self.weight_matrix[:,:,0,:,k])
      x = tf.expand_dims(x, axis=-1)

      #Converting NHWDC format
      receptive_field = tf.cast(tf.constant(np.ones((3,3,3,1,1))), tf.float32)
      output = tf.nn.conv3d(x, receptive_field, strides=1, padding="SAME")
      outputs.append(output)
    outputs = tf.concat(outputs, axis=-1)
    return outputs
